In [26]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('../data/raw/VN_housing_dataset.csv')

cols_to_drop = ['Unnamed: 0', 'Ngày', 'Địa chỉ', 'Dài', 'Rộng']
df = df.drop(columns=cols_to_drop)

df.columns = [
    'District', 'Ward', 'House_type', 'Legal', 'Floors', 'Bedrooms', 'Area', 'Price_per_m2'
]

# Hàm làm sạch số (loại bỏ chữ, đổi dấu phẩy thành chấm)
def extract_number(value):
    if pd.isna(value): return np.nan
    # Chuyển về chuỗi thường
    text = str(value).lower()
    # Nếu có dấu phẩy (86,96) thay bằng chấm (86.96)
    text = text.replace(',', '.')
    # Tìm số (bao gồm cả số thập phân)
    match = re.search(r"[-+]?\d*\.\d+|\d+", text)
    if match:
        return float(match.group())
    return np.nan

# Áp dụng cho các cột số
cols_to_clean = ['Area', 'Price_per_m2', 'Bedrooms', 'Floors']
for col in cols_to_clean:
    df[col] = df[col].apply(extract_number)

# Tính TỔNG GIÁ TRỊ CĂN NHÀ (Đây là cột quan trọng nhất để dự đoán)
# Công thức: Giá/m2 * Diện tích = Tổng giá (đơn vị: Triệu VNĐ)
df['Total_Price_Billion'] = (df['Price_per_m2'] * df['Area']) / 1000
# (Chia 1000 để đổi từ Triệu sang Tỷ cho số nhỏ dễ nhìn)

# Xử lý cột Quận/Huyện (Bỏ chữ 'Quận', 'Phường')
df['District'] = df['District'].str.replace('Quận', '').str.strip()
df['Ward'] = df['Ward'].str.replace('Phường', '').str.replace('Xã', '').str.strip()

# Xử lý thiếu (Missing Values)
# Xóa dòng nếu thiếu Diện tích, Giá, Số tầng và số phòng ngủ
df = df.dropna(subset=['Area', 'Total_Price_Billion', 'Floors', 'Bedrooms'])

# Cột Pháp lý: Điền 'Dang_cap_nhat'
df['Legal'] = df['Legal'].fillna('Dang_cap_nhat')


# Lọc theo Diện tích
# Chỉ lấy nhà từ 10m2 đến 500m2
df = df[(df['Area'] >= 10) & (df['Area'] <= 500)]

# Lọc bỏ số phòng ngủ
# Nếu diện tích < 40m2 mà có >= 8 phòng ngủ -> XÓA (Vì quá chật)
df = df[~((df['Area'] < 40) & (df['Bedrooms'] >= 8))]

# Nếu số tầng < 2 mà có >= 5 phòng ngủ (trừ khi diện tích > 100m2) -> XÓA
df = df[~((df['Floors'] < 2) & (df['Bedrooms'] >= 5) & (df['Area'] < 100))]

# Lọc theo Giá
# Chỉ lấy nhà giá từ 500 Triệu (0.5 tỷ) đến 100 Tỷ
df = df[(df['Total_Price_Billion'] >= 0.5) & (df['Total_Price_Billion'] <= 100)]

# Lưu file sạch
df.to_csv('../data/processed/clean_vn_housing.csv', index=False)

In [27]:
df.count()

District               36128
Ward                   36122
House_type             36128
Legal                  36128
Floors                 36128
Bedrooms               36128
Area                   36128
Price_per_m2           36128
Total_Price_Billion    36128
dtype: int64

In [28]:
df.head(10)

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion
0,Cầu Giấy,Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4.0,5.0,46.0,86.96,4.00016
2,Hai Bà Trưng,Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,65.00,2.60000
7,Hai Bà Trưng,Đống Mác,"Nhà mặt phố, mặt tiền",Đã có sổ,6.0,5.0,32.0,184.38,5.90016
9,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ,4.0,3.0,41.0,64.63,2.64983
11,Hoàng Mai,Định Công,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,30.0,83.33,2.49990
15,Long Biên,Bồ Đề,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,52.0,93.27,4.85004
17,Hoàn Kiếm,Phúc Tân,"Nhà mặt phố, mặt tiền",Đã có sổ,7.0,10.0,165.0,103.03,16.99995
18,Long Biên,Gia Thụy,"Nhà mặt phố, mặt tiền",Đã có sổ,2.0,3.0,70.0,102.86,7.20020
20,Nam Từ Liêm,Phương Canh,"Nhà ngõ, hẻm",Đã có sổ,5.0,3.0,32.0,51.56,1.64992
21,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ,5.0,6.0,65.0,75.38,4.89970


In [29]:
# 1. Lưu và mã hóa District
df['District_old'] = df['District'].copy()
df['District'] = pd.factorize(df['District'])[0]

print("--- BẢNG TRA CỨU DISTRICT ---")
district_map = df[['District_old', 'District']].drop_duplicates().sort_values('District')
print(district_map.to_string(index=False)) # Dùng to_string để hiện thị đẹp hơn

--- BẢNG TRA CỨU DISTRICT ---
    District_old  District
        Cầu Giấy         0
    Hai Bà Trưng         1
         Hà Đông         2
       Hoàng Mai         3
       Long Biên         4
       Hoàn Kiếm         5
     Nam Từ Liêm         6
 Huyện Thanh Trì         7
         Đống Đa         8
         Ba Đình         9
      Thanh Xuân        10
     Bắc Từ Liêm        11
          Tây Hồ        12
  Huyện Hoài Đức        13
   Huyện Gia Lâm        14
 Huyện Chương Mỹ        15
  Thị xã Sơn Tây        16
  Huyện Đông Anh        17
   Huyện Mê Linh        18
Huyện Thường Tín        19
 Huyện Thanh Oai        20
   Huyện Sóc Sơn        21
Huyện Đan Phượng        22
  Huyện Phúc Thọ        23
  Huyện Quốc Oai        24
Huyện Thạch Thất        25


In [30]:
# 2. Lưu và mã hóa Ward
df['Ward_old'] = df['Ward'].copy()
df['Ward'] = pd.factorize(df['Ward'])[0]

print("--- BẢNG TRA CỨU WARD ---")
ward_map = df[['Ward_old', 'Ward']].drop_duplicates().sort_values('Ward')
print(ward_map.to_string(index=False))

--- BẢNG TRA CỨU WARD ---
           Ward_old  Ward
                NaN    -1
           Nghĩa Đô     0
          Minh Khai     1
           Đống Mác     2
           Văn Quán     3
          Định Công     4
              Bồ Đề     5
           Phúc Tân     6
           Gia Thụy     7
        Phương Canh     8
          Tương Mai     9
             La Khê    10
             Tây Mỗ    11
       Tả Thanh Oai    12
            Láng Hạ    13
           Tam Hiệp    14
            Cống Vị    15
          Bách Khoa    16
          Vĩnh Phúc    17
       Khương Trung    18
           Trung Tự    19
          Kiến Hưng    20
          Ô Chợ Dừa    21
           Mai Dịch    22
      Hoàng Văn Thụ    23
           Vạn Phúc    24
        Thanh Lương    25
            Ngọc Hà    26
             Phú Đô    27
          Long Biên    28
           Kim Liên    29
          Phúc Diễn    30
             Kim Mã    31
           Cầu Diễn    32
          Trung Văn    33
        Thượng Đình    34
          Ng

In [31]:
# 3. Lưu và mã hóa House_type
df['House_type_old'] = df['House_type'].copy()
df['House_type'] = pd.factorize(df['House_type'])[0]

print("--- BẢNG TRA CỨU HOUSE_TYPE ---")
house_type_map = df[['House_type_old', 'House_type']].drop_duplicates().sort_values('House_type')
print(house_type_map.to_string(index=False))

--- BẢNG TRA CỨU HOUSE_TYPE ---
       House_type_old  House_type
         Nhà ngõ, hẻm           0
Nhà mặt phố, mặt tiền           1
         Nhà biệt thự           2
      Nhà phố liền kề           3


In [32]:
# 4. Lưu và mã hóa Legal
df['Legal_old'] = df['Legal'].copy()
df['Legal'] = pd.factorize(df['Legal'])[0]

print("--- BẢNG TRA CỨU LEGAL ---")
legal_map = df[['Legal_old', 'Legal']].drop_duplicates().sort_values('Legal')
print(legal_map.to_string(index=False))


--- BẢNG TRA CỨU LEGAL ---
    Legal_old  Legal
     Đã có sổ      0
  Đang chờ sổ      1
Dang_cap_nhat      2
 Giấy tờ khác      3


In [33]:
df

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion,District_old,Ward_old,House_type_old,Legal_old
0,0,0,0,0,4.0,5.0,46.0,86.96,4.00016,Cầu Giấy,Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ
2,1,1,0,0,4.0,4.0,40.0,65.00,2.60000,Hai Bà Trưng,Minh Khai,"Nhà ngõ, hẻm",Đã có sổ
7,1,2,1,0,6.0,5.0,32.0,184.38,5.90016,Hai Bà Trưng,Đống Mác,"Nhà mặt phố, mặt tiền",Đã có sổ
9,2,3,0,0,4.0,3.0,41.0,64.63,2.64983,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ
11,3,4,0,0,5.0,4.0,30.0,83.33,2.49990,Hoàng Mai,Định Công,"Nhà ngõ, hẻm",Đã có sổ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82471,8,56,3,0,5.0,6.0,50.0,290.00,14.50000,Đống Đa,Nam Đồng,Nhà phố liền kề,Đã có sổ
82472,10,70,3,0,4.0,4.0,42.0,71.43,3.00006,Thanh Xuân,Khương Đình,Nhà phố liền kề,Đã có sổ
82474,9,118,0,0,3.0,3.0,32.0,375.00,12.00000,Ba Đình,Phúc Xá,"Nhà ngõ, hẻm",Đã có sổ
82479,8,13,1,0,5.0,4.0,35.0,94.29,3.30015,Đống Đa,Láng Hạ,"Nhà mặt phố, mặt tiền",Đã có sổ


In [34]:
df = df.drop(columns=['District_old', 'Ward_old', 'House_type_old', 'Legal_old'])

In [35]:
df

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion
0,0,0,0,0,4.0,5.0,46.0,86.96,4.00016
2,1,1,0,0,4.0,4.0,40.0,65.00,2.60000
7,1,2,1,0,6.0,5.0,32.0,184.38,5.90016
9,2,3,0,0,4.0,3.0,41.0,64.63,2.64983
11,3,4,0,0,5.0,4.0,30.0,83.33,2.49990
...,...,...,...,...,...,...,...,...,...
82471,8,56,3,0,5.0,6.0,50.0,290.00,14.50000
82472,10,70,3,0,4.0,4.0,42.0,71.43,3.00006
82474,9,118,0,0,3.0,3.0,32.0,375.00,12.00000
82479,8,13,1,0,5.0,4.0,35.0,94.29,3.30015


In [36]:
# Lưu file sạch
df.to_csv('../data/processed/clean_vn_housing.csv', index=False)